# Binary classification of commutative diagrams
## 1. Data pipeline

In [17]:
import pandas as pd
import numpy as np
import os
import shutil
import random
import math
from tensorflow.keras.preprocessing import image as img

### 1.1 Expand working directory with folders for training images

In [28]:
## Set unsorted directory name here
unsortedSamplesDirName = 'png-2021-all_contexts'
##

cwd = os.getcwd()

unsortedSamplesDir = os.path.join(cwd, unsortedSamplesDirName)
assert('positive' in os.listdir(unsortedSamplesDir) and 'negative' in os.listdir(unsortedSamplesDir))
unsortedPositiveSamplesDir = os.path.join(unsortedSamplesDir, 'positive')
unsortedNegativeSamplesDir = os.path.join(unsortedSamplesDir, 'negative')

sortedSamplesDir = os.path.join(cwd, 'sortedSamples')
os.makedirs(sortedSamplesDir, exist_ok=True)

trainDir = os.path.join(sortedSamplesDir, 'train')
testDir = os.path.join(sortedSamplesDir, 'test')
valDir = os.path.join(sortedSamplesDir, 'val')
os.makedirs(trainDir, exist_ok=True)
os.makedirs(testDir, exist_ok=True)
os.makedirs(valDir, exist_ok=True)

trainPositiveDir = os.path.join(trainDir, 'positive')
trainNegativeDir = os.path.join(trainDir, 'negative')
testPositiveDir = os.path.join(testDir, 'positive')
testNegativeDir = os.path.join(testDir, 'negative')
valPositiveDir = os.path.join(valDir, 'positive')
valNegativeDir = os.path.join(valDir, 'negative')
os.makedirs(trainPositiveDir, exist_ok=True)
os.makedirs(trainNegativeDir, exist_ok=True)
os.makedirs(testPositiveDir, exist_ok=True)
os.makedirs(testNegativeDir, exist_ok=True)
os.makedirs(valPositiveDir, exist_ok=True)
os.makedirs(valNegativeDir, exist_ok=True)

### 1.2 Copy images into train, test and validation folders
The code currently randomly truncates the greater partition between positive and negative to achieve parity.

In [29]:
## Set train- and test set relative size here, as well as desired ratio of positive samples
trainRatio = 0.6
testRatio = 0.2
valRatio = 0.2
assert(trainRatio + testRatio + valRatio == 1.)

positiveRatio = 0.5 # Desired ratio of positive samples in the sorted data
##


positiveSamplesFilenameList = os.listdir(unsortedPositiveSamplesDir)
negativeSamplesFilenameList = os.listdir(unsortedNegativeSamplesDir)
random.shuffle(positiveSamplesFilenameList)
random.shuffle(negativeSamplesFilenameList)
numPositiveSamples = len(positiveSamplesFilenameList)
numNegativeSamples = len(negativeSamplesFilenameList)

if numPositiveSamples > numNegativeSamples:
    positiveSamplesFilenameList = positiveSamplesFilenameList[:numNegativeSamples]
    numPositiveSamples = len(positiveSamplesFilenameList)
elif numNegativeSamples > numPositiveSamples:
    negativeSamplesFilenameList = negativeSamplesFilenameList[:numPositiveSamples]
    numNegativeSamples = len(negativeSamplesFilenameList)
assert(numPositiveSamples == numNegativeSamples)
    
numSamples = numPositiveSamples + numNegativeSamples

numTrainSamples = math.floor(numSamples*trainRatio)
numTestSamples = math.floor(numSamples*testRatio)
numValSamples = math.floor(numSamples*valRatio)


unsortedSamplesInfo = {'posDir':unsortedPositiveSamplesDir, 'negDir':unsortedNegativeSamplesDir, 'posFilenameList':positiveSamplesFilenameList,
                       'negFilenameList':negativeSamplesFilenameList}

trainCopyInfo = {'num':numTrainSamples, 'posDir':trainPositiveDir, 'negDir':trainNegativeDir}
testCopyInfo = {'num':numTestSamples, 'posDir':testPositiveDir, 'negDir':testNegativeDir}
valCopyInfo = {'num':numValSamples, 'posDir':valPositiveDir, 'negDir':valNegativeDir}

def copyImagesInPartition(unsortedSamplesInfo:dict, partitionCopyInfo:dict):
    for _ in range(math.floor(partitionCopyInfo['num']*positiveRatio)):
        copyImage(unsortedSamplesInfo['posFilenameList'], unsortedSamplesInfo['posDir'], partitionCopyInfo['posDir'])
    for _ in range(math.floor(partitionCopyInfo['num']*(1. - positiveRatio))):
        copyImage(unsortedSamplesInfo['negFilenameList'], unsortedSamplesInfo['negDir'], partitionCopyInfo['negDir'])

def copyImage(sampleFilenameList, srcDir, dstDir):
    filename = sampleFilenameList.pop()
    src = os.path.join(srcDir, filename)
    dst = os.path.join(dstDir, filename)
    try:
        shutil.copyfile(src, dst)
    except PermissionError: # Ignores straggler files such as notebook checkpoints
        pass

sampleDirectorySizes = [len(directory) for directory in [
    os.listdir(trainPositiveDir), os.listdir(trainNegativeDir), os.listdir(testPositiveDir), os.listdir(testNegativeDir),
    os.listdir(testPositiveDir), os.listdir(testNegativeDir)]]

if all(size == 0 for size in sampleDirectorySizes):
    copyImagesInPartition(unsortedSamplesInfo, trainCopyInfo)
    copyImagesInPartition(unsortedSamplesInfo, testCopyInfo)
    copyImagesInPartition(unsortedSamplesInfo, valCopyInfo)
else:
    raise Exception("Sorted image directories are not empty.")

# TRASH: Is this actually more readable than the functions above?
"""
if all(size == 0 for size in sampleDirectorySizes):
    for _ in range(math.floor(numTrainSamples*positiveRatio)):
        filename = positiveSamplesFilenameList.pop()
        src = os.path.join(unsortedPositiveSamplesDir, filename)
        dst = os.path.join(trainPositiveDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numTrainSamples*(1. - positiveRatio))):
        filename = negativeSamplesFilenameList.pop()
        src = os.path.join(unsortedNegativeSamplesDir, filename)
        dst = os.path.join(trainNegativeDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numTestSamples*positiveRatio)):
        filename = positiveSamplesFilenameList.pop()
        src = os.path.join(unsortedPositiveSamplesDir, filename)
        dst = os.path.join(testPositiveDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numTestSamples*(1. - positiveRatio))):
        filename = negativeSamplesFilenameList.pop()
        src = os.path.join(unsortedNegativeSamplesDir, filename)
        dst = os.path.join(testNegativeDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numValSamples*positiveRatio)):
        filename = positiveSamplesFilenameList.pop()
        src = os.path.join(unsortedPositiveSamplesDir, filename)
        dst = os.path.join(valPositiveDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numValSamples*(1. - positiveRatio))):
        filename = negativeSamplesFilenameList.pop()
        src = os.path.join(unsortedNegativeSamplesDir, filename)
        dst = os.path.join(valNegativeDir, filename)
        shutil.copyfile(src, dst)
else:
    raise Exception("Sorted image directories are not empty.")"""

'\nif all(size == 0 for size in sampleDirectorySizes):\n    for _ in range(math.floor(numTrainSamples*positiveRatio)):\n        filename = positiveSamplesFilenameList.pop()\n        src = os.path.join(unsortedPositiveSamplesDir, filename)\n        dst = os.path.join(trainPositiveDir, filename)\n        shutil.copyfile(src, dst)\n\n    for _ in range(math.floor(numTrainSamples*(1. - positiveRatio))):\n        filename = negativeSamplesFilenameList.pop()\n        src = os.path.join(unsortedNegativeSamplesDir, filename)\n        dst = os.path.join(trainNegativeDir, filename)\n        shutil.copyfile(src, dst)\n\n    for _ in range(math.floor(numTestSamples*positiveRatio)):\n        filename = positiveSamplesFilenameList.pop()\n        src = os.path.join(unsortedPositiveSamplesDir, filename)\n        dst = os.path.join(testPositiveDir, filename)\n        shutil.copyfile(src, dst)\n\n    for _ in range(math.floor(numTestSamples*(1. - positiveRatio))):\n        filename = negativeSamplesFile

[*Optional*]: Test whether a sample handful of the images were copied to the correct folder

In [30]:
## Set number of images to be checked per folder here
comparisonSampleSize = 50
imageFiletype = 'png'
##

# These must be redefined here since they were popped
positiveSamplesFilenameList = os.listdir(unsortedPositiveSamplesDir)
negativeSamplesFilenameList = os.listdir(unsortedNegativeSamplesDir)

sortedTrainPositiveFilenameList = os.listdir(trainPositiveDir)
random.shuffle(sortedTrainPositiveFilenameList)
sortedTrainNegativeFilenameList = os.listdir(trainNegativeDir)
random.shuffle(sortedTrainNegativeFilenameList)
sortedTestPositiveFilenameList = os.listdir(testPositiveDir)
random.shuffle(sortedTestPositiveFilenameList)
sortedTestNegativeFilenameList = os.listdir(testNegativeDir)
random.shuffle(sortedTestNegativeFilenameList)
sortedValPositiveFilenameList = os.listdir(valPositiveDir)
random.shuffle(sortedValPositiveFilenameList)
sortedValNegativeFilenameList = os.listdir(valNegativeDir)
random.shuffle(sortedValNegativeFilenameList)

allPositiveFilenameLists = [sortedTrainPositiveFilenameList, sortedTestPositiveFilenameList, sortedValPositiveFilenameList]
allNegativeFilenameLists = [sortedTrainNegativeFilenameList, sortedTestNegativeFilenameList, sortedValNegativeFilenameList]

def matchesImageFiletype(sampleFilename : str, imageFiletype : str): # Needed to ignore straggler files such as notebook checkpoints
    return sampleFilename[:len(imageFiletype)] == imageFiletype

for fList in allPositiveFilenameLists:
    assert(all(sampleFilename in positiveSamplesFilenameList for sampleFilename in fList[:comparisonSampleSize] if matchesImageFiletype(sampleFilename, imageFiletype)))
for fList in allNegativeFilenameLists:
    assert(all(sampleFilename in negativeSamplesFilenameList for sampleFilename in fList[:comparisonSampleSize] if matchesImageFiletype(sampleFilename, imageFiletype)))

In [ ]:
imageDataGenerator = img.ImageDataGenerator(rescale=1./255)
dataGenerator = imageDataGenerator.flow_from_directory(
    
)